In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "You're a helpful assistant. Answer questions using only the follwing context. If you don't know about the answer just say you don't know, don't make it up:\n{context}"),
  ("human", "{question}")
])

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

# stuff type chain


In [38]:
chain.invoke("Describe Victory Mansions.")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is described as having glass doors at the entrance, which allow gritty dust to enter. The hallway of Victory Mansions has a smell of boiled cabbage and old rag mats. There is a large colored poster on one end of the hallway, depicting the face of a man in his forties with a black mustache. The flat where Winston lives is located on the seventh floor, and the elevator is often out of service due to the electricity being cut off during daylight hours as part of an economy drive. The building is not well-maintained, with rotting houses and patched windows in the surrounding area.')